<a href="https://colab.research.google.com/github/Igorvl/Projects/blob/master/%D0%9E%D0%B1%D1%83%D1%87%D0%B0%D1%8E%D1%89%D0%B0%D1%8F%2C_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B8_%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8_%D0%9F%D0%B5%D1%80%D0%B5%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9D%D0%A1_%22%D0%9F%D0%B0%D1%81%D1%81%D0%B0%D0%B6%D0%B8%D1%80%D1%8B%20%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81%D0%B0%22_Ultra_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используя базу "Пассажиры автобуса", подготовьте данные для обучения нейронной сети, классифицирующей изображение на два класса:
- входящий пассажир
- выходящий пассажир

Добейтесь точности работы модели на проверочной выборке не ниже 85%

Ссылка на датасет: https://storage.yandexcloud.net/aiueducation/Content/base/l4/bus.zip

In [ ]:
# Ваше решение
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gdown
import os
import pdb 
import zipfile
from PIL import Image
%matplotlib inline

# скачивание, распаковывка, если данных нет
if not os.path.exists('/content/Входящий') or not os.path.exists('/content/Выходящий'):

  gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l4/bus.zip', None, quiet = True)

  with zipfile.ZipFile('bus.zip', 'r') as zip_file:
      zip_file.extractall('/content')

In [ ]:
# считывание входящих, сбор в Array
imLst = []
for x in os.listdir('/content/Входящий/'):
  img = Image.open(f'/content/Входящий/{x}')
  img =img.resize(size=(128,64))
  imArr = np.array(img)
  imLst.append(imArr)
baseIn = np.array(imLst)

# считывание выходящих, сбор в Array
imLst = []
for x in os.listdir('/content/Выходящий/'):
  img = Image.open(f'/content/Выходящий/{x}')
  img =img.resize(size=(128,64))
  imArr = np.array(img)
  imLst.append(imArr)
baseOut = np.array(imLst)

In [ ]:
# нормализация
x_baseIn = baseIn.reshape(baseIn.shape[0],-1).astype('float')/255
x_baseOut = baseOut.reshape(baseOut.shape[0],-1).astype('float')/255
y_baseIn = np.ones(x_baseIn.shape[0])
y_baseOut = np.zeros(x_baseOut.shape[0])
print(x_baseIn.shape)
print(x_baseOut.shape)
print(y_baseIn.shape)
print(y_baseOut.shape)

(6485, 24576)
(2596, 24576)
(6485,)
(2596,)


In [ ]:
# объединение массивов входящих и выходящих
x_base = np.concatenate([x_baseIn, x_baseOut])
y_base = np.concatenate([y_baseIn, y_baseOut])
print(x_base.shape)
print(y_base.shape)

(9081, 24576)
(9081,)


In [ ]:
# Разбиение наборов на общую и тестовую выборки
x_all, x_test, y_all, y_test = train_test_split(x_base, y_base, test_size=0.1, shuffle=True, random_state = 42)

# Разбиение общей выборки на обучающую и проверочную 
x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, test_size=0.1, shuffle=True, random_state = 42)
print(f'Обучающая выборка изображений {len(x_train)}')
print(f'Обучающая выборка меток {len(y_train)}')
print(f'Проверочная выборка изображений {len(x_val)}')
print(f'Проверочная выборка меток {len(y_val)}')
print(f'Тестовая выборка изображений {len(x_test)}')
print(f'Тестовая выборка меток {len(y_test)}')



Обучающая выборка изображений 7354
Обучающая выборка меток 7354
Проверочная выборка изображений 818
Проверочная выборка меток 818
Тестовая выборка изображений 909
Тестовая выборка меток 909


In [ ]:
# ф-я сбора модели
TYPES = 1
def modelBuild(d1=100, d2=20, drp=0, bNorm=False, activ='relu'):
  model = Sequential()
  model.add(Dense(d1, input_dim=x_train.shape[1], activation=activ))
  model.add(Dropout(drp))
  model.add(Dense(d2, activation=activ))
  if bNorm: 
    model.add(BatchNormalization())
  model.add(Dense(TYPES, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
  return model
model = modelBuild()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2457700   
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                2020      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,459,741
Trainable params: 2,459,741
Non-trainable params: 0
_________________________________________________________________


In [ ]:

bestAccur,val = 0,pd.Series(dtype=pd.StringDtype())

# первый слой
for i in [1024]:
# второй слой
  for j in [128]:

        model = modelBuild(i, j, 0, False, activ='relu')
        hist = model.fit(x_train, y_train, batch_size=20, 
                         validation_data=(x_val, y_val),epochs=15, verbose=1)
        evalHist = model.evaluate(x_test, y_test, verbose=1)
        if bestAccur < evalHist[1]:
          bestAccur = evalHist[1]
          val = [evalHist[1],evalHist[0],i,j]
          # pdb.set_trace()   
          print(val)           
          
print(f'\n\nМаксимальная точность классификации на тестовой выборке {val[0]*100}% c loss - {val[1]} при параметрах:\nПервый Dense слой {val[2]}\nВторой Dense слой {val[3]}')

Epoch 1/15
368/368 [==============================] - 57s 154ms/step - loss: 1.4523 - accuracy: 0.6904 - val_loss: 0.4766 - val_accuracy: 0.7433
Epoch 2/15
368/368 [==============================] - 58s 157ms/step - loss: 0.4946 - accuracy: 0.7691 - val_loss: 0.4332 - val_accuracy: 0.8068
Epoch 3/15
368/368 [==============================] - 60s 162ms/step - loss: 0.4714 - accuracy: 0.7852 - val_loss: 0.4949 - val_accuracy: 0.7995
Epoch 4/15
368/368 [==============================] - 55s 149ms/step - loss: 0.4502 - accuracy: 0.7932 - val_loss: 0.4002 - val_accuracy: 0.8289
Epoch 5/15
368/368 [==============================] - 55s 151ms/step - loss: 0.4249 - accuracy: 0.8103 - val_loss: 0.3803 - val_accuracy: 0.8350
Epoch 6/15
368/368 [==============================] - 58s 157ms/step - loss: 0.3888 - accuracy: 0.8287 - val_loss: 0.5012 - val_accuracy: 0.7824
Epoch 7/15
368/368 [==============================] - 57s 154ms/step - loss: 0.3572 - accuracy: 0.8397 - val_loss: 0.3371 - val_ac